# F3 Connecting the Dots

Now that you have implemented the QFT for one and two qubits, we will focus on implementing a generalized QFT on $n$  qubits. To get an intuition behind the gates involved, it is easier to look at the action of the QFT on the computational basis states, rather than the matrix formulation.

Author: [Monit Sharma](https://github.com/MonitSharma)
LinkedIn: [Monit Sharma](https://www.linkedin.com/in/monitsharma/)
Twitter: [@MonitSharma1729](https://twitter.com/MonitSharma1729)
Medium : [MonitSharma](https://medium.com/@_monitsharma)

The following figure shows the QFT circuit on $n$ qubits, with a focus on the action of rotations on the first qubit.

![](https://codebook.xanadu.ai/pics/f3-circuit-opt3.png)

### Codercise F.3.1. 
Implement the QFT for three qubits.



In [ ]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def three_qubit_QFT(basis_id):
    """A circuit that computes the QFT on three qubits.
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
        
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    qml.BasisStatePreparation(bits, wires=[0, 1, 2])
    
    ##################
    # YOUR CODE HERE #
    ################## 
    qml.Hadamard(wires=0)
    qml.ControlledPhaseShift(np.pi/2, wires=[1,0])
    qml.ControlledPhaseShift(np.pi/2, wires=[2,0])

    qml.Hadamard(wires=1)
    qml.ControlledPhaseShift(np.pi/2, wires=[2,1])
    qml.Hadamard(wires=2)
    qml.SWAP(wires=[0,2])
    return qml.state()

    


### Codercise F.3.2. 
Implement a circuit that reverses the order of  qubits using SWAP gates. You need only  gates for this.



In [ ]:
dev = qml.device('default.qubit', wires=4)

            
def swap_bits(n_qubits):
    """A circuit that reverses the order of qubits, i.e.,
    performs a SWAP such that [q1, q2, ..., qn] -> [qn, ... q2, q1].
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    i = 0
    j = n_qubits -1
    while i != j and i < j:
	    qml.SWAP(wires=[i,j])
	    i += 1 
	    j -= 1


@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    # qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()


Now we will implement rotations:

![](https://codebook.xanadu.ai/pics/f3-circuit-opt2.png)

### Codercise F.3.3. 
Implement the circuit that performs the Hadamards and controlled rotations on  qubits using `qml.ControlledPhaseShift`. The `swap_bits` operation defined in the exercise above is available to use. The result is the n-qubit QFT!

In [ ]:
dev = qml.device('default.qubit', wires=4)

def qft_rotations(n_qubits):
    """A circuit performs the QFT rotations on the specified qubits.
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """

    ##################
    # YOUR CODE HERE #
    ################## 
    for i in range(n_qubits):
	    qml.Hadamard(wires=i)
	    for j in range(i+1, n_qubits):
		    qml.ControlledPhaseShift(np.pi/2**(j-i), wires =[j,i])

    

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()


You have successfully implemented the QFT! We will now try to implement a recursive solution to the rotations part of QFT as well. Let's take a look at the circuit again.

![](https://codebook.xanadu.ai/pics/f3-circuit-opt2.png)

### Codercise F.3.4. 
Implement the circuit that performs the Hadamard operations and controlled rotations on  qubits recursively. The swap_bits operation defined earlier is available for you

In [ ]:
dev = qml.device('default.qubit', wires=4)

def qft_recursive_rotations(n_qubits, wire=0):
    """A circuit that performs the QFT rotations on the specified qubits
        recursively.
        
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
        wire (int): An integer identifying the wire 
                    (or the qubit) to apply rotations on.
    """

    ##################
    # YOUR CODE HERE #
    ################## 
    if wire == n_qubits - 1:
	    qml.Hadamard(wires=wire)
	    return
    else:
	    qml.Hadamard(wires=wire)
	    for j in range(wire+1,n_qubits):
		    qml.ControlledPhaseShift(np.pi/2**(j - wire), wires=[j,wire])
	    qft_recursive_rotations(n_qubits, wire+1)

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    qft_recursive_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()


### Codercise F.3.5. 
Implement the QFT using qml.QFT.

In [ ]:
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def pennylane_qft(basis_id, n_qubits):
    """A that circuit performs the QFT using PennyLane's QFT template.
    
    Args:
        basis_id (int): An integer value identifying 
            the basis state to construct.
        n_qubits (int): An integer identifying the 
            number of qubits.
            
    Returns:
        array[complex]: The state after applying the QFT to the qubits.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))

    ##################
    # YOUR CODE HERE #
    ################## 
    qml.QFT(wires=range(n_qubits))
    return qml.state()
